In [ ]:
import pandas as pd
import json

In [ ]:
price_history = pd.read_json('price_collection.json')
price_history.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close']
price_history.sort_values('Date', ascending=True, inplace=True)

with open('lotsize.json', "r") as read_file:
    lotsize = json.load(read_file)
    read_file.close()

black_list = ('DSKY','LNZL','LNZLP', 'ARSA', 'ASSB', 'AVAN', 'BISVP', 'BRZL', 'BSPBP', 'CHGZ', 'DZRD', 'DZRDP', 'EELT', 'ELTZ', 'GEMA', 'IGST', 'IGSTP', 'JNOS', 'JNOSP', 'KBSB', 'KCHE', 'KCHEP', 'KGKC', 'KGKCP', 'KMEZ', 'KOGK', 'KRKN', 'KROTP', 'KRSB', 'KRSBP', 'KTSB', 'KTSBP', 'KUBE', 'KUZB', 'LPSB', 'LVHK', 'MAGE', 'MAGEP', 'MFGS', 'MFGSP', 'MGNZ', 'MGTS', 'MISB', 'MISB', 'MISBP', 'MRKK', 'MRSB', 'NFAZ', 'NKSH', 'NNSB', 'NNSBP', 'OMZZP', 'PRMB', 'RBCM', 'RDRB', 'RGSS', 'ROSB', 'RTGZ', 'RTSB', 'RTSBP', 'RUSI', 'RZSB', 'SAGO', 'SAGOP', 'SARE', 'SAREP', 'SIBG', 'SLEN', 'STSB', 'STSBP', 'SVET', 'TASB', 'TASBP', 'TGKDP', 'TNSE', 'TORS', 'TORSP', 'TUZA', 'UCSS', 'UKUZ', 'URKZ', 'USBN', 'UTAR', 'VGSB', 'VGSBP', 'VJGZ', 'VJGZP', 'VLHZ', 'VRSBP', 'WTCM', 'WTCMP', 'YKEN', 'YKENP', 'YRSB', 'YRSBP', 'ZILL')

In [ ]:
#FMBA ver2
idea_end = []
idea_active = []
idea_ready = []
for share in price_history.Ticker.unique():
    #select data for each ticker
    period = price_history[(price_history['Ticker'] == share) & (price_history['Date'] > '2022-11-01')]
    #searching high volume over 10 ema
    #create a short-list to count ema10 volume
    volume_check = period['Volume'].to_frame()
    volume_check['Volume_EMA10'] = volume_check['Volume'].ewm(span=10).mean()
    volume_check['increase'] = volume_check['Volume'] / volume_check['Volume_EMA10']
    #reverse array to find last high-volume min x2 over ema10 at green day
    volume_check = volume_check[:9:-1]
    volume_check = volume_check[volume_check['increase'] > 2]
    first_day = 0

    for indx in volume_check.index:
        if period.at[indx, 'Close'] > period.at[indx, 'Open'] * 1.08:
            first_day = indx
            break
    if first_day > 0:        
        #counting emas
        ema_atr = period['Close'].to_frame()
        ema_atr['price_EMA'] = ema_atr['Close'].ewm(span=10).mean()
        #counting ATR
        for i,tek in enumerate(ema_atr.index):
            if i == 0:
                ema_atr.at[tek,'TR'] = period.at[tek,'High'] - period.at[tek,'Low']
            else:
                pred = ema_atr.index[i-1]
                tr1 = period.at[tek,'High'] - period.at[tek,'Low']
                tr2 = abs(period.at[tek,'High'] - period.at[pred,'Close'])
                tr3 = abs(period.at[pred,'Close'] - period.at[tek,'Low'])
                ema_atr.at[tek,'TR'] = max(tr1, tr2, tr3)
        ema_atr['ATR'] = ema_atr['TR'].ewm(alpha=1/14).mean()
        
        #check strategy
        buy_price = 0
        sell_price = 0
        for i,indx in enumerate(ema_atr.index):
            if indx > first_day+300:                
                if buy_price == 0:
                    if (period.at[indx, 'Low'] <= ema_atr.at[indx, 'price_EMA'] * 1.003 ):
                        lot_trade = period.at[indx, 'Volume'] / lotsize.get(share)
                        if lot_trade > 10000:                            
                            buy_price = ema_atr.at[indx, 'price_EMA'] * 1.003
                            buy_date = period.at[indx, 'Date']                            
                elif sell_price == 0:
                    prev = ema_atr.index[i-1]
                    take_profit = period.at[prev, 'Close'] + 1.5 * ema_atr.at[prev, 'ATR']
                    stop_loss = period.at[prev, 'Close'] - 0.5 * ema_atr.at[prev, 'ATR']
                    if period.at[indx, 'High'] >= take_profit:
                        sell_price = take_profit
                        sell_date = period.at[indx, 'Date']
                    elif period.at[indx, 'Low'] <= stop_loss:
                        sell_price = stop_loss
                        sell_date = period.at[indx, 'Date']
        #sort ideas by status
        if sell_price > 0:
            idea_res = round((sell_price/buy_price-1)*100, 2)
            idea_end.append([share, buy_date, sell_date, idea_res])
        elif buy_price > 0:
            idea_res = round((ema_atr.at[indx, 'Close']/buy_price-1)*100, 2)
            idea_active.append([buy_date, share, idea_res, round(take_profit,5), round(stop_loss,5)])
        else:
            lot_trade = period.at[indx, 'Volume'] / lotsize.get(share)
            if lot_trade > 10000:
                idea_ready.append(share)
#print(idea_end)
for i in idea_active:
    print(i)
print(idea_ready)

In [ ]:
mas = [0.049, 0.099, 0.124, 0.141, 0.165, 0.199, 0.249, 0.332, 0.499, 0.999]
for posi in mas:
    res = 1000000
    depo = res
    depo_fact = res
    active_posi = {}
    period = price_history[price_history['Date'] > '2022-11-01']
    for d in period.Date.unique():
        for share in idea_end:        
            if share[2] == d:
                tick_posi = active_posi.pop(share[0],0)            
                depo += tick_posi * share[3]/100
                depo_fact += tick_posi * (1+share[3]/100)
            elif share[1] == d and share[0] not in black_list:
                tick_posi = posi * depo
                if tick_posi <= depo_fact:
                    depo_fact -= tick_posi
                    active_posi[share[0]] = tick_posi
        if posi == 0.49:
            print(active_posi)
    print(posi, round(depo_fact,2), round(depo_fact/res,2))    